In [1]:
import sys
sys.executable

'/Library/Developer/CommandLineTools/usr/bin/python3'

In [2]:
! /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install -U faiss-cpu transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 6.5 MB/s eta 0:00:0000:0100:01


In [2]:
from unlimiformer import UnlimiformerArguments, Unlimiformer

In [9]:
from transformers import BartForConditionalGeneration, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

modelname = "abertsch/unlimiformer-bart-govreport-alternating"
model = BartForConditionalGeneration.from_pretrained(modelname)

In [10]:
example_input = """Howth has been settled since prehistoric times, and features in Irish mythology. A fishing village and small trading port from at least the 14th century, Howth has grown to become a busy and affluent suburb of Dublin, with a mix of suburban residential development, wild hillside and heathland, golf courses, cliff and coastal paths, a small quarry and a busy commercial fishing port. The only neighbouring district on land is Sutton. Howth is also home to one of the oldest occupied buildings in Ireland, Howth Castle, and its estate.
After Brian Ború, the High King of Ireland, defeated the Norse in 1014, many Norse fled to Howth to regroup and remained a force until their final defeat in Fingal in the middle of the 11th century. Howth still remained under the control of Irish and localised Norse forces until the invasion of Ireland by the Anglo-Normans in 1169.

Without the support of either the Irish or Scandinavian powers, Howth was isolated and fell to the Normans in 1177. One of the victorious Normans, Armoricus (or Almeric) Tristram, was granted much of the land between the village and Sutton. According to the historian Samuel Lewis:

In 1177, Sir Amorey Tristram and Sir John de Courcy landed here at the head of a large military force, and totally defeated the Danish inhabitants in a sanguinary battle at the bridge of Evora, over a mountain stream which falls into the sea near the Baily lighthouse. This victory secured to Sir Amorey the lordship of Howth, of which his descendants have continued in possession to the present day, under the name of St. Laurence, which Almaric, third baron, assumed in fulfilment of a vow previously to his victory over the Danes near Clontarf, in a battle fought on the festival of that saint. The territory of Howth was confirmed to Almaric de St. Laurence by King John....

On 26 July 1914, 900 rifles were landed at Howth by Erskine Childers for the Irish Volunteers. Many were used against the British in the Easter Rising and in the subsequent Anglo-Irish War. Among the members of the Howth branches of the Irish Volunteers and Cumann na mBan who participated in this event were the well-known writers Padraic Colum and Mary Colum. Members of both the Howth Volunteers and Baldoyle section of the Irish Citizen Army participated in the Easter Rising in Dublin city and in Fingal. A strong local branch of Sinn Féin developed in the area and there was considerable local involvement in both the Irish War of Independence and Irish Civil War.

Howth Head is one of the dominant features of Dublin Bay, with a number of peaks, the highest of which is Black Linn. In one area, near Shielmartin, there is a small peat bog, the "Bog of the Frogs". The wilder parts of Howth can be accessed by a network of paths (many are rights of way) and much of the centre and east is protected as part of a Special Area of Conservation of 2.3 square kilometres (570 acres), as well as by a Special Amenity Area Order.

The peninsula has a number of small, fast-running streams, three of which run through the village, with more, including the Bloody Stream, in the adjacent Howth Demesne. The streams passing through the village are, from east to west, Coulcour Brook (falling to Balscadden Bay), Gray's Brook or the Boggeen Stream (falling to the eastern end of the harbour), and Offington Stream (passing under Findlater's to the western side of the harbour). Other streams are met along the cliff walks, including the Whitewater Brook, with a tributary in a sunken area of plants and ponds, and then the Balsaggart Stream.

More than half of Howth Head, and of the Howth area, totalling around 1,500 acres, is subject to a Special Amenity Area Order, a provision of Irish law designed to protect areas of natural beauty or biodiversity. Prepared by the local authority, after a consultative process under a community environmental process known as SEMPA,[27] and formally proposed by Brendan Howlin as a minister, the order was developed in cooperation with a unit of Trinity College Dublin. It was made in 1999, confirmed in 2000, and is subject to 5-yearly reviews, the most recent having been conducted in 2015.[28] At the time of its making the majority of the lands covered formed part of the Howth Estate, with the other significant landowners being developers Treasury Holdings (85 hectares) and Gerry Gannon (33 hectares), and Howth Golf Club (48 hectares). The Council stated that the order was needed to protect the environment of the designated area by restricting development there, while recognising the need "to encourage tourism-related developments in the remainder of Howth".
"""

example = tokenizer(example_input, truncation=False, return_tensors="pt")
truncated_example = tokenizer(example_input, truncation=True, max_length=1024, return_tensors="pt")

print(f"INPUT LENGTH (tokens): {example['input_ids'].shape[-1]}")



Token indices sequence length is longer than the specified maximum sequence length for this model (1042 > 1024). Running this sequence through the model will result in indexing errors


INPUT LENGTH (tokens): 1042


In [11]:

defaults = UnlimiformerArguments()
unlimiformer_kwargs = {
            'layer_begin': defaults.layer_begin, 
            'layer_end': defaults.layer_end,
            'unlimiformer_head_num': defaults.unlimiformer_head_num, 
            'exclude_attention': defaults.unlimiformer_exclude, 
            'chunk_overlap': defaults.unlimiformer_chunk_overlap,
            'model_encoder_max_len': defaults.unlimiformer_chunk_size,
            'verbose': defaults.unlimiformer_verbose, 'tokenizer': tokenizer,
            'unlimiformer_training': defaults.unlimiformer_training,
            'use_datastore': defaults.use_datastore,
            'flat_index': defaults.flat_index,
            'test_datastore': defaults.test_datastore,
            'reconstruct_embeddings': defaults.reconstruct_embeddings,
            'gpu_datastore': defaults.gpu_datastore,
            'gpu_index': defaults.gpu_index
}



In [12]:
unlimited_model = Unlimiformer.convert_model(model, **unlimiformer_kwargs)

In [13]:
unlimited_model.eval()

In [14]:
unlimiformer_out = tokenizer.batch_decode(
    unlimited_model.generate(**example, max_length=512), 
    ignore_special_tokens=True)

unlimiformer_out 

/Users/alvas/Library/Python/3.8/lib/python/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['</s><s>Howth Castle, in Dublin, Ireland, is a hamlet and small trading port located on the west coast of Ireland. More than half of the area is protected by a Special Amenity Area Order, a provision of Irish law designed to protect areas of natural or biodiversity.</s>']

In [16]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [15]:
truncated_out = tokenizer.batch_decode(
    model.generate(**truncated_example, max_length=512))

truncated_out

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/alvas/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-89716c5ea316>", line 2, in <module>
    model.generate(**truncated_example, max_length=512))
  File "/Users/alvas/Library/Python/3.8/lib/python/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/Users/alvas/Library/Python/3.8/lib/python/site-packages/transformers/generation/utils.py", line 1604, in generate
    return self.beam_search(
  File "/Users/alvas/Library/Python/3.8/lib/python/site-packages/transformers/generation/utils.py", line 2902, in beam_search
    outputs = self(
  File "/Users/alvas/Library/Python/3.8/lib/python/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/Users/alvas/Library/Python/3.8/lib/python/si

TypeError: object of type 'NoneType' has no len()